In [46]:
import json
import random
import copy
from datetime import timedelta, datetime
import os
from pathlib import Path
import configs
import sys 

sys.path.insert(0, os.path.dirname(Path(__file__).parent.absolute()))
import configs as configs

In [39]:
historydata = json.load(open('../original_data_sample/weatherapi/history/2023-05-26/1.json'))
input_dir = configs.global_params['temp_dir']

In [55]:
forecast_data = {}

start_date = datetime(2014, 1, 1)
end_date = datetime(2017,12,31)
day = timedelta(days=1)

date_inst = start_date
while date_inst < end_date:

    year_start = date_inst.year

    original_date = (date_inst - datetime(year_start, 1,1)).days
    min_date = max(0,original_date - 5)
    max_date = min(364, original_date + 5)
    date = random.randint(min_date, max_date)

    forecast_data['location'] = historydata['location']
    current_hour = random.randint(8,14)
    current_weather = copy.deepcopy(historydata['forecast']['forecastday'][date]['hour'][current_hour])

    current_weather['last_updated_epoch'] = current_weather['time_epoch']
    current_weather['last_updated'] =  current_weather['time']
    del current_weather["time_epoch"]
    del current_weather["time"]
    del current_weather["windchill_c"]
    del current_weather["windchill_f"]
    del current_weather["heatindex_c"]
    del current_weather["heatindex_f"]
    del current_weather["dewpoint_c"]
    del current_weather["dewpoint_f"]
    del current_weather["will_it_rain"]
    del current_weather["chance_of_rain"]
    del current_weather["will_it_snow"]
    del current_weather["chance_of_snow"]

    forecast_data['current'] = current_weather

    forecast_data['forecast'] = {
        'forecastday' : [{}]
    }
    forecast_data['forecast']['forecastday'][0]['date'] = historydata['forecast']['forecastday'][date]['date']
    forecast_data['forecast']['forecastday'][0]['date_epoch'] = historydata['forecast']['forecastday'][date]['date_epoch']

    forecast_data['forecast']['forecastday'][0]['day'] = historydata['forecast']['forecastday'][date]['day']
    forecast_data['forecast']['forecastday'][0]['day']['daily_will_it_rain'] = historydata['forecast']['forecastday'][date]['hour'][current_hour]['will_it_rain']
    forecast_data['forecast']['forecastday'][0]['day']['daily_chance_of_rain'] = historydata['forecast']['forecastday'][date]['hour'][current_hour]['chance_of_rain']
    forecast_data['forecast']['forecastday'][0]['day']['daily_will_it_snow'] = historydata['forecast']['forecastday'][date]['hour'][current_hour]['will_it_snow']
    forecast_data['forecast']['forecastday'][0]['day']['daily_chance_of_snow'] = historydata['forecast']['forecastday'][date]['hour'][current_hour]['chance_of_snow']
    forecast_data['forecast']['forecastday'][0]['day']['totalsnow_cm'] = random.randint(0,50) 

    forecast_data['forecast']['forecastday'][0]['astro'] = historydata['forecast']['forecastday'][date]['astro']
    forecast_data['forecast']['forecastday'][0]['astro']['is_moon_up'] = 0
    forecast_data['forecast']['forecastday'][0]['astro']['is_sun_up'] = 1
    
    hours =  historydata['forecast']['forecastday'][date]['hour']

    for hr in hours:
        time = datetime.strptime( hr['time'], '%Y-%m-%d %H:%M')

        new_time = time.replace(year=year_start)
        hr['time'] = new_time.strftime("%Y-%m-%d %H:%M")
        hr['time_epoch'] = new_time.timestamp()

    forecast_data['forecast']['forecastday'][0]['hour'] = hours

    str_dt = date_inst.strftime('%Y-%m-%d')

    os.mkdir(f'{Path.home()}/{input_dir}/weatherapi/forecast/{str_dt}')

    with open(f'{Path.home()}/{input_dir}/weatherapi/forecast/{str_dt}/1.json', 'w') as fp:
        json.dump(forecast_data, fp, indent=4)
    date_inst += day